# Mastering SQL Queries with BigQuery: Data Exploration and Analysis

#### Introduction

In this kernel, we embark on a journey of data exploration and analysis using the power of **BigQuery**, Google's cloud-based data warehouse. Our focus will be on **mastering SQL queries** to uncover valuable insights from complex datasets.

This project delves into the **"Catalonian Mobile Coverage"** dataset, a rich source of information on mobile network coverage, operator activity, and user behavior. The primary dataset, **mobile_data_2015_2017_cleaned**, has undergone thorough cleaning and normalization to ensure data quality.

#### Key Data Sources:


- Main dataset: **mobile_data_2015_2017_cleaned**
    - Provides insights into mobile network coverage, operator activity, and user behavior.

- Additional datasets: **cat_percapita_income_by_province_2015_2017** and **cat_pop_by_province_2015_2017**
 - Supply data on per capita income and population by province.


#### Prominent SQL Commands and Functions:

   - **SQL SELECT Statements**: Querying data, filtering, sorting, and retrieving relevant information.
   - **Grouping and Aggregation**: Summarizing data with COUNT, SUM, AVG, and more.
   - **Window Functions**: Calculating rankings, tracking changes, and gaining insights.
   - **Conditional Statements**: Categorizing data using CASE statements.
   - **Subqueries**: Creating complex queries with embedded subqueries.
   - **Views**: Organizing and filtering data for in-depth analysis.
   - **Statistical Aggregate Functions**: Computing correlations, covariances, quartiles, and more.
   - **Date Functions**: Exploring date ranges, identifying top activity dates, and analyzing monthly and hourly patterns.
   - **Plotting and Visualization**: Presenting insights through interactive plots and visualizations.

#### Some Key Insights We Aim to Uncover:

- Identifying the top net_provider in Catalonia with the highest activity recorded.
- Finding the operator with the highest activity.
- Determining which net_provider/operator combination exhibits the highest average signal strength.
- Dividing operators into quartiles based on signal strength to assess performance variations.
- Analyzing changes in signal strength and net_provider speed within specific nets.
- Ranking net_provider and operators by record count and exploring the impact on rankings based on signal strength.


### Import libraries and modules

In [1]:
import pandas as pd
from google.cloud import bigquery
from IPython.display import display
from IPython.display import Markdown

import matplotlib.pyplot as plt
import seaborn as sns

### Import function: Interactive SQL Query to Pandas DataFrame Converter

Import the custom query_df and run_query functions from the 'query_functions.py' file to execute SQL queries using a pre-configured BigQuery client object.

In [2]:
from query_functions import query_df  # Execute the query and return the output as a DataFrame
from query_functions import run_query  # Execute the query without returning a DataFrame, used for INSERT, UPDATE, DELETE, etc.

### Datasets and Tables paths

In [3]:
# Catalonian mobile coverage (2015-2017)
mobile_data_cleaned = "bq-analyst-230590.project_cat_mobile_coverage_2015_2017.mobile_data_2015_2017_cleaned"

# Per capita income by catalan province (2015-2017)
percapita_income = "bq-analyst-230590.project_cat_mobile_coverage_2015_2017.cat_percapita_income_by_province_2015_2017"

# Catalan population and density by province (2015-2017)
pop_density = "bq-analyst-230590.project_cat_mobile_coverage_2015_2017.cat_pop_by_province_2015_2017"

### Preview datasets

Mobile Data Cleaned

In [4]:
# SQL query:
query = f"""
    SELECT *
    FROM `{mobile_data_cleaned}`
    LIMIT 100
    """

# Execute the query and store the result
mob_data = query_df(query)

# Show table
display(mob_data.head(),mob_data.tail())
print("Shape: \n", mob_data.shape)

,date,hour,lat,long,signal,net_provider,operator,description,net,speed,satellites,precision,position_provider,activity,postal_code,town_name,position_geom,province,year
0,2015-09-08,23:27:34,41.73223,2.74127,12,EE,EE,STATE_EMERGENCY_ONLY,2G,74.2,3.0,13.0,GPS,IN_VEHICLE,082845,Tordera,POINT(2.74127 41.73223),Barcelona,2015
1,2015-09-08,09:18:28,41.98149,2.78145,15,EE,EE,STATE_IN_SERVICE,Undefined net,131.4,8.0,47.0,GPS,IN_VEHICLE,171557,Salt,POINT(2.78145 41.98149),Girona,2015
2,2015-09-09,16:03:41,41.95650,2.78532,9,EE,EE,STATE_EMERGENCY_ONLY,2G,144.2,10.0,15.0,GPS,IN_VEHICLE,172155,Vilablareix,POINT(2.78532 41.9565),Girona,2015
3,2015-10-30,17:49:22,41.44253,2.23031,15,O2,O2,STATE_EMERGENCY_ONLY,4G,83.0,15.0,6.0,GPS,STILL,080155,Badalona,POINT(2.23031 41.44253),Barcelona,2015
4,2015-10-30,17:15:22,41.35004,2.14897,15,O2,O2,STATE_EMERGENCY_ONLY,4G,1.9,7.0,7.0,GPS,STILL,080193,Barcelona,POINT(2.14897 41.35004),Barcelona,2015


,date,hour,lat,long,signal,net_provider,operator,description,net,speed,satellites,precision,position_provider,activity,postal_code,town_name,position_geom,province,year
95,2015-04-25,20:02:58,41.43535,2.20704,13,Movistar,ONO,STATE_EMERGENCY_ONLY,3G,28.2,2.0,4.0,GPS,IN_VEHICLE,080193,Barcelona,POINT(2.20704 41.43535),Barcelona,2015
96,2015-12-12,17:24:37,41.37828,2.18022,21,Movistar,ONO,STATE_EMERGENCY_ONLY,3G,16.6,3.0,9.0,GPS,ON_BICYCLE,080193,Barcelona,POINT(2.18022 41.37828),Barcelona,2015
97,2015-04-01,23:44:58,41.40605,2.17749,18,Movistar,ONO,STATE_IN_SERVICE,3G,0.3,0.0,47.0,GPS,TILTING,080193,Barcelona,POINT(2.17749 41.40605),Barcelona,2015
98,2015-05-15,05:50:37,41.39605,2.20159,14,Movistar,ONO,STATE_EMERGENCY_ONLY,3G,21.4,5.0,2.0,GPS,IN_VEHICLE,080193,Barcelona,POINT(2.20159 41.39605),Barcelona,2015
99,2015-05-29,15:33:39,41.40565,2.20613,14,Movistar,ONO,STATE_EMERGENCY_ONLY,3G,114.3,4.0,7.0,GPS,IN_VEHICLE,080193,Barcelona,POINT(2.20613 41.40565),Barcelona,2015


Shape: 
 (100, 19)


In [5]:
# List of columns
mob_data.columns

Index(['date', 'hour', 'lat', 'long', 'signal', 'net_provider', 'operator',
       'description', 'net', 'speed', 'satellites', 'precision',
       'position_provider', 'activity', 'postal_code', 'town_name',
       'position_geom', 'province', 'year'],
      dtype='object')

In [6]:
# Column types
mob_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 19 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   date               100 non-null    object 
 1   hour               100 non-null    object 
 2   lat                100 non-null    float64
 3   long               100 non-null    float64
 4   signal             100 non-null    int64  
 5   net_provider       100 non-null    object 
 6   operator           100 non-null    object 
 7   description        100 non-null    object 
 8   net                100 non-null    object 
 9   speed              100 non-null    float64
 10  satellites         100 non-null    float64
 11  precision          100 non-null    float64
 12  position_provider  100 non-null    object 
 13  activity           100 non-null    object 
 14  postal_code        100 non-null    object 
 15  town_name          100 non-null    object 
 16  position_geom      100 non-

As previously mentioned in the earlier kernel, during the retrieval of 'date' and 'hour' from Google BigQuery for the 'SQL Data Cleaning For Mobile Coverage Dataset', these fields were transformed to 'object' types. However, it's important to note that in the original Google BigQuery dataset, their formats are 'date' and 'time', respectively. If there is a need to update these formats in the original table, the required queries are provided below:

    (Please be aware that while we are displaying this data as a Python DataFrame for reference, the actual changes should be implemented in the original BigQuery dataset. We will directly query the original dataset, and we won't be utilizing this DataFrame for further analysis.)

In [7]:
# SQL query: in BigQuery,convert 'date' to date format and 'hour' to time format
query = f"""
     -- Convert 'date' column to date data type
    UPDATE `{mobile_data_cleaned}`
    SET date = CAST(date AS DATE)
    """
# Execute the query
# run_query(query)


query = f"""
     -- Convert 'date' column to date data type
    UPDATE `{mobile_data_cleaned}`
    SET hour = CAST(hour AS TIME)
    """

# Execute the query
# run_query(query)

In [8]:
# Summary Statistics of numerical features
mob_data.describe()

,lat,long,signal,speed,satellites,precision,year
count,100.000000,100.000000,100.000000,100.000000,100.00000,100.00000,100.0
mean,41.432138,2.197444,16.520000,20.632000,3.66000,15.68000,2015.0
std,0.113194,0.126069,7.506872,32.402693,3.04916,14.04026,0.0
min,41.329690,1.821740,2.000000,0.000000,0.00000,2.00000,2015.0
25%,41.393655,2.158917,11.000000,2.475000,1.00000,8.00000,2015.0
50%,41.404620,2.194935,14.000000,8.550000,4.00000,11.00000,2015.0
75%,41.417075,2.206485,22.000000,21.250000,5.00000,21.00000,2015.0
max,41.981490,2.785320,63.000000,157.100000,15.00000,87.00000,2015.0


Per Capita Income & Population Density Tables

In [63]:
# Datasets: {mobile_data_cleaned}, {pop_density}, {percapita_income}

# SQL query:
query = f"""
SELECT
     -- Columns from 'percapita_income' c table
    c.year,
    c.province,
    c.per_capita_income,
     -- Columns from 'pop_density' p table
    p.population,
    p.sq_km,
    p.density_per_sq_km,
FROM `{percapita_income}` c
JOIN `{pop_density}` p
    ON c.year = p.year AND c.province = p.province
    """

# Execute the query
query_df(query)

,year,province,per_capita_income,population,sq_km,density_per_sq_km
0,2015,Lleida,20136,742138,12172,61.0
1,2015,Tarragona,22486,811089,6303,128.7
2,2015,Barcelona,27214,5618162,7726,727.2
3,2015,Girona,25200,765783,5908,129.6
4,2016,Tarragona,23130,810947,6303,128.7
5,2016,Lleida,20713,742099,12172,61.0
6,2016,Barcelona,27913,5635085,7726,729.4
7,2016,Girona,25598,766273,5908,129.7
8,2017,Barcelona,28481,5652301,7726,731.6
9,2017,Lleida,21091,741884,12172,61.0


### Execute queries:

**1. Filtering and Sorting**

**2. Aggregation**

**3. Join Operations**

**4. Left, Right, Inner and Full Join**

**8. Case Statements**

**9. Statistical Aggregate Functions**

**10. Window Functions**

**11. Datetime Functions**

**12. Brief Exploratory Data Analysis**

#### 1. Filtering and Sorting

**1.1. Retrieve all records from the '{mobile_2015_2017}' table for the year 2016 where the 'signal' strength is above a threshold of 100 and 'net' is not 'Undefined net'.**

**SQL Query Structure and Functions Utilized:**

- **Clauses**: SELECT, FROM, WHERE, AND, ORDER BY, DESC, LIMIT
- **Operators**: =, >, !=


In [11]:
# Datasets: {mobile_data_cleaned}, {pop_density}, {percapita_income}

# SQL Query:
query = f"""
    SELECT 
        year,
        operator,
        net,
        signal
    FROM `{mobile_data_cleaned}`
    WHERE 
        year = 2016
        AND signal > 50
        AND NET != 'Undefined net'
    ORDER BY signal DESC
    LIMIT 1000
    """

# Execute the query
query_df(query)  

,year,operator,net,signal
0,2016,Jazztel,3G,99
1,2016,Orange,3G,99
2,2016,Movistar,2G,99
3,2016,Movistar,3G,99
4,2016,Movistar,2G,99
...,...,...,...,...
995,2016,Orange,4G,65
996,2016,Vodafone,4G,65
997,2016,Movistar,4G,65
998,2016,Movistar,4G,65


**1.2. Retrieve records from the '{mobile_2015_2017}' table for the year 2017 where 'net' is '4G' and 'description' is 'STATE_IN_SERVICE,' and for the year 2016 where 'net' is '3G' and 'signal' strength exceeds the year's average '3G' signal strength.**

**SQL Query Structure and Functions Utilized:**

- **Clauses**: SELECT, FROM, WHERE, OR, AND, ORDER BY, LIMIT
- **Aggregated Funtions**: AVG()
- **Operators**: =, >
- **Subqueries**

In [12]:
# Datasets: {mobile_data_cleaned}, {pop_density}, {percapita_income}

# SQL Query:
query = f"""
    SELECT
      year,
      operator,
      net,
      description,
      signal,
      speed
    FROM `{mobile_data_cleaned}`
    WHERE
        -- Condition 1: Select records for '4G' net in 2017 with STATE_IN_SERVICE description and speed > 100.0
      (year = 2017
      AND net = '4G'
      AND description = 'STATE_IN_SERVICE'
      AND speed > 100.0)
      OR
        -- Condition 2: Select records for 3G network in 2016 with signal greater than the average signal strength for '3G' records in 2016
      (year = 2016
      AND net = '3G'
      AND signal > (
      
           -- Subquery to calculate average signal strength for '3G' records in 2016
          SELECT
              AVG(signal)
          FROM `{mobile_data_cleaned}`
          WHERE
              year = 2016
              AND net = '3G')
       )
    ORDER BY 1 DESC
    LIMIT 1000;
    """
# Execute the query
query_df(query)    

,year,operator,net,description,signal,speed
0,2017,Pepephone,4G,STATE_IN_SERVICE,10,126.7
1,2017,Orange,4G,STATE_IN_SERVICE,12,108.3
2,2017,Vodafone,4G,STATE_IN_SERVICE,26,105.7
3,2017,Masmovil,4G,STATE_IN_SERVICE,8,193.0
4,2017,Movistar,4G,STATE_IN_SERVICE,18,106.2
...,...,...,...,...,...,...
995,2016,Yoigo,3G,STATE_EMERGENCY_ONLY,22,3.4
996,2016,Simyo,3G,STATE_EMERGENCY_ONLY,16,0.1
997,2016,Yoigo,3G,STATE_EMERGENCY_ONLY,16,14.9
998,2016,Yoigo,3G,STATE_EMERGENCY_ONLY,16,6.4


**1.3. Calculate and display the top 3 Orange 'operators' with the highest 'activity count' for the '4G' category, for both the years 2017 and 2016. Organize the results by the number of 'activity counts' in descending order and present the findings.**

**SQL Query Structure and Functions Utilized:**

- **Clauses**: WITH, SELECT, FROM, WHERE, AND, GROUP BY, ORDER BY, DESC
- **Aggregated Functions**: COUNT()
- **Set Operations**: UNION ALL
- **Operators**: =
- **CTEs**: OrangeOperators_4G_2017, OrangeOperators_4G_2016


In [13]:
# Datasets: {mobile_data_cleaned}, {pop_density}, {percapita_income}

# SQL Query:
query = f"""
WITH OrangeOperators_4G_2017 AS ( 

     -- Common Table Expression (CTE): Top-3 activity count '4G' Orange Operator in 2017
    SELECT
        year,
        net,
        operator,
        COUNT(*) activity_count
    FROM `{mobile_data_cleaned}`
    WHERE
      net_provider = 'Orange'
      AND net = '4G'
      AND year = 2017
    GROUP BY 1,2,3 
    ORDER BY 1 DESC, 2 DESC, 4 DESC
    LIMIT 3
    ),
    
     -- CTE: Top-3 activity count '4G' Orange Operator in 2016
    OrangeOperators_4G_2016 AS ( 
    SELECT
        year,
        net,
        operator,
        COUNT(*) activity_count
    FROM `{mobile_data_cleaned}`
    WHERE
      net_provider = 'Orange'
      AND net = '4G'
      AND year = 2016
    GROUP BY 1,2,3 
    ORDER BY 1 DESC, 2 DESC, 4 DESC
    LIMIT 3
    )

SELECT
    year,
    net,
    operator,
    activity_count
FROM `OrangeOperators_4G_2017`
UNION ALL
SELECT
    year,
    net,
    operator,
    activity_count
FROM `OrangeOperators_4G_2016`
ORDER BY 1 DESC, 4 DESC
    """
# Execute the query
query_df(query) 

,year,net,operator,activity_count
0,2017,4G,Orange,173994
1,2017,4G,adamo,13295
2,2017,4G,Jazztel,11513
3,2016,4G,Orange,533644
4,2016,4G,Jazztel,108857
5,2016,4G,Simyo,15286


**1.4. Calculate the percentage of records where the activity is 'IN_VEHICLE' and 'net' is NOT 'Undefined net' in the '{mobile_2015_2017}' table for the year 2017.**

**SQL Query Structure and Functions Utilized:**

- **Clauses**: SELECT, FROM, WHERE, AND, GROUP BY
- **Aggregated Functions**: COUNT()
- **Mathematical Functions**: ROUND()
- **Operators**: /, =, !=
- **Subqueries**: in_vehicle_subquery, total_subquery


In [14]:
# Datasets: {mobile_data_cleaned}, {pop_density}, {percapita_income}

# SQL Query:
query = f"""
SELECT
    net,
    ROUND((in_vehicle_2017_count / total_count * 100), 2) AS in_vehicle_2017_perc
FROM (
        -- Subquery to calculate the count of records GROUPED BY 'net'      
    SELECT 
        net,
        COUNT(*) AS in_vehicle_2017_count
    FROM `{mobile_data_cleaned}`
    WHERE year = 2017
        AND activity = 'IN_VEHICLE'
        AND net != 'Undefined net'
    GROUP BY 1
    ) in_vehicle_subquery,
    
        -- Subquery to calculate the total count of records.
    (
     SELECT COUNT(*) AS total_count
     FROM `{mobile_data_cleaned}`
     WHERE year = 2017
        AND activity = 'IN_VEHICLE'
        AND net != 'Undefined net'
    ) total_subquery
    """
# Execute the query
query_df(query)   

,net,in_vehicle_2017_perc
0,3G,26.18
1,4G,57.83
2,2G,15.99


#### 2. Aggregation

**2.1. Calculate the count, average, standard deviation, minimum, and maximum speed for each province in Catalonia with a population density of less than 130 inhabitants per square kilometer and a name containing 'ona'. What are the speed statistics for these provinces?**

**SQL Query Structure and Functions Utilized:**

- **Clauses**: SELECT, FROM, WHERE, GROUP BY, HAVING
- **Aggregated Functions**: COUNT(), ROUND(), MIN(), MAX()
- **Statistical Functions**: STDDEV_POP()
- **String Functions**: REGEXP_CONTAINS()
- **Operators**: =, <
- **Joins**: JOIN

In [15]:
# Datasets: {mobile_data_cleaned}, {pop_density}, {percapita_income}

# SQL Query:
query = f"""
    SELECT
        m.province,
        COUNT(m.speed) speed_counts,
        ROUND(AVG(m.speed),2) avg_speed,
        ROUND(STDDEV_POP(m.speed),2) std_speed,
        MIN(m.speed) min_speed,
        MAX(m.speed) max_speed
    FROM `{mobile_data_cleaned}` m
    JOIN `{pop_density}` p
     ON m.province = p.province
    WHERE p.density_per_sq_km < 130 # This would include 'Girona', 'Tarragona' and 'Lleida'
    GROUP BY 1
     -- Use of REGEXP_CONTAINS() to find 'ona' pattern in 'province' column names
     HAVING REGEXP_CONTAINS(m.province, 'ona') # This would include 'Barcelona', 'Girona' and 'Tarragona'
    """
# Execute the query
query_df(query)

,province,speed_counts,avg_speed,std_speed,min_speed,max_speed
0,Girona,4128609,26.64,34.75,0.0,255.0
1,Tarragona,2974152,27.76,37.08,0.0,254.9


_As there is no population or per capita income data available for 2014, we use the `CAST` function to convert the numeric result of the growth calculation to a string. This ensures that the 'No data' placeholder, which is a string, can be used consistently in the result. Without the `CAST`, there could be a data type mismatch in the `IFNULL` function,
which requires consistent data types for its arguments, ensuring that 'No data' is returned when data is missing._

**2.2. For net_providers with multiple operators, find the total, maximum, and average number of satellites used by each unique pair of network providers (net_provider) and operators (operator).**

**Provide the total satellites usage, maximum satellites usage within a single activity, and average satellites usage in the result for these specific network providers.**

**SQL Query Structure and Functions Utilized:**

- **Clauses:** SELECT, FROM, WHERE, GROUP BY, HAVING, ORDER BY
- **Aggregated Functions:** SUM(), MAX(), AVG(), COUNT(), DISTINCT()
- **Conversion Functions:** CAST()
- **String Functions:** CONCAT()
- **Operators:** >
- **Subqueries:** unique_pairs_subquery


In [18]:
# Datasets: {mobile_data_cleaned}

# SQL Query:
query = f"""
SELECT
 netprovider_operator,
 total_satellites_usage,
 max_satellites_per_record,
 avg_satellites_per_record
FROM (
     -- Create unique pairs of net_provider + operator...
    SELECT
        CONCAT(net_provider, ' - ', operator) AS netprovider_operator,
        CAST(SUM(satellites) AS INT64) AS total_satellites_usage,
        CAST(MAX(satellites) AS INT64) AS max_satellites_per_record,
        CAST(AVG(satellites) AS INT64) AS avg_satellites_per_record
    FROM `{mobile_data_cleaned}`
    WHERE net_provider IN (
            
         -- ... only if the net_provider has multiple operators
        SELECT net_provider
        FROM `{mobile_data_cleaned}`
        GROUP BY net_provider
        HAVING COUNT(DISTINCT operator) > 1
        )
    GROUP BY 1
) AS unique_pairs_subquery
ORDER BY total_satellites_usage DESC, max_satellites_per_record DESC
    """

# Execute the query
query_df(query)

,netprovider_operator,total_satellites_usage,max_satellites_per_record,avg_satellites_per_record
0,Movistar - Movistar,21025335,42,5
1,Vodafone - Vodafone,13025235,29,5
2,Orange - Orange,9709569,29,5
3,Orange - Jazztel,2369254,26,6
4,Movistar - Pepephone,1850320,25,5
5,Orange - Simyo,615420,25,5
6,Orange - RACC,586762,25,4
7,Movistar - Tuenti,214929,22,5
8,Orange - PARLEM,207948,22,5
9,Movistar - ONO,196699,20,4


**2.3. Find the first three recorded activities in terms of date and hour for each net_provider. Include the date, hour, and net_provider columns in the result.**

**SQL Query Structure and Functions Utilized:**

- **Clauses**: SELECT, FROM, WHERE, IS NOT NULL, GROUP BY, ORDER BY, ASC, LIMIT
- **Aggregate Functions**: MIN()


In [17]:
# Datasets: {mobile_data_cleaned}

# SQL Query:
query = f"""
    SELECT
        date,
        hour,
        net_provider AS first_net_provider
    FROM `{mobile_data_cleaned}`
    WHERE net_provider IS NOT NULL
    GROUP BY 1,2,3
    ORDER BY MIN(date), hour ASC
    LIMIT 3
    """
# Execute the query
query_df(query)


,date,hour,first_net_provider
0,2015-01-05,00:48:37,Vodafone
1,2015-01-05,11:27:20,Vodafone
2,2015-01-05,11:30:57,Vodafone


**2.4. Find the first recorded activity in terms of date and hour for each province. Include the date, hour, and net_provider columns in the result.**

**SQL Query Structure and Functions Utilized:**

- **Clauses**: SELECT, FROM, WHERE, IS NOT NULL, ORDER BY, ASC

- **Operators:** =

- **Subqueries**: ranked_data

- **Window Functions**: ROW_NUMBER()


In [19]:
# Datasets: {mobile_data_cleaned}

# SQL Query:
query = f"""
    SELECT
        province,
        date,
        hour,
        net_provider AS first_net_provider
    FROM (
         -- Subquery to assign a row number to each record based on the order of 'date' and 'hour' within each 'province'
        SELECT
            date,
            hour,
            net_provider,
            province,
             -- The ROW_NUMBER() function partitions data by 'province,' arranging it chronologically and assigns sequential numbers to rows within each province.
            ROW_NUMBER() OVER (PARTITION BY province ORDER BY date, hour) AS row_num
        FROM `{mobile_data_cleaned}`
        WHERE net_provider IS NOT NULL
    ) ranked_data
     -- Selects records with row numbers equal to 1 (the earliest recorded activity for each 'province')
    WHERE row_num = 1
    ORDER BY date ASC
    """
# Execute the query
query_df(query)

,province,date,hour,first_net_provider
0,Barcelona,2015-01-05,00:48:37,Vodafone
1,Tarragona,2015-01-15,16:56:58,Vodafone
2,Girona,2015-01-25,11:45:18,Vodafone
3,Lleida,2015-01-29,10:58:50,Vodafone


#### 3. SQL Joins: A Variety of Techniques

**3.1. Join the '{mobile_2015_2017}' table with the '{pop_density}' and the '{pop_density}' tables to retrieve records from all tables for the year 2016 and calculate the average signal strength (signal) for each 'province'.**

**SQL Query Structure and Functions Utilized:**

- **Clauses**: SELECT, FROM, WHERE, AND, GROUP BY, ORDER BY, DESC
- **Aggregated Functions**: AVG()
- **Mathematical Functions**: ROUND()
- **Operators:** =
- **Joins**: JOIN


In [20]:
# Datasets: {mobile_data_cleaned}, {pop_density}, {percapita_income}

# SQL Query:
query = f"""
    SELECT 
      m.province,
      p.sq_km,
      p.population AS population_2016,
      p.density_per_sq_km AS pop_density_2016,
      c.per_capita_income AS per_capita_inc_2016,
      ROUND(AVG(m.signal),2) avg_signal_2016
    FROM `{mobile_data_cleaned}` m
    JOIN `{pop_density}` p 
      ON m.province = p.province AND m.year = p.year
    JOIN `{percapita_income}` c
      ON m.province = c.province AND m.year = c.year
    WHERE m.year = 2016
    GROUP BY 1,2,3,4,5
    ORDER BY 6 DESC;
    """
# Execute the query
query_df(query) 

,province,sq_km,population_2016,pop_density_2016,per_capita_inc_2016,avg_signal_2016
0,Barcelona,7726,5635085,729.4,27913,13.38
1,Lleida,12172,742099,61.0,20713,12.83
2,Tarragona,6303,810947,128.7,23130,12.69
3,Girona,5908,766273,129.7,25598,11.95


**3.2. Calculate the average signal strength (signal) and average monthly activity for each month using data from the dataset {mobile_data_cleaned}, and order the results in descending order based on the average monthly activity.**

**SQL Query Structure and Functions Utilized:**

- **Clauses**: SELECT, WITH, FROM, GROUP BY, ORDER BY, DESC
- **Aggregated Functions**: COUNT(), AVG() 
- **Date Functions**: EXTRACT()
- **Mathematical Functions**: ROUND()
- **Operators**: =, *, /
- **CTEs**: MonthlyActivity, PercentTotalActivity
- **Joins**: JOIN, CROSS JOIN


In [21]:
# Datasets: {mobile_data_cleaned}, {pop_density}, {percapita_income}

# SQL Query:
query = f"""
     -- CTE: Calculate monthly registered activity and average signal
    WITH MonthlyActivity AS (
        SELECT
            EXTRACT(MONTH FROM date) as month,
            COUNT(*) AS monthly_activity,
            ROUND(AVG(signal), 2) as avg_signal
        FROM `{mobile_data_cleaned}`
        GROUP BY 1
    ),

     -- CTE: Calculate the percentage of monthly activity for the total annual activity
    PercentTotalActivity AS (
        SELECT
            ma.month,
            ROUND((ma.monthly_activity / t.total_activity),2) * 100 as perc_total_activity
        FROM MonthlyActivity ma
        CROSS JOIN (
        
             -- The CROSS JOIN combines the row count from 'MonthlyActivity' with the total record count 
             -- to compute annual activity. This links each monthly count with the overall record count        
        
            SELECT
                COUNT(*) AS total_activity
            FROM `{mobile_data_cleaned}`
        ) t
    )

    SELECT 
        ma.month,
        pta.perc_total_activity,
        ROUND(AVG(ma.monthly_activity), 2) AS avg_monthly_activity,
        ma.avg_signal
    FROM MonthlyActivity ma
    JOIN PercentTotalActivity pta
     ON ma.month = pta.month
    GROUP BY 1, 2, 4
    ORDER BY 3 DESC
    """

# Execute the corrected query
query_df(query)

,month,perc_total_activity,avg_monthly_activity,avg_signal
0,9,10.0,1065753.0,13.14
1,5,10.0,1037712.0,13.21
2,6,9.0,990234.0,13.06
3,3,9.0,968422.0,13.34
4,10,9.0,925787.0,13.32
5,7,9.0,915300.0,13.08
6,8,8.0,872794.0,12.75
7,12,8.0,865982.0,13.27
8,11,8.0,857949.0,13.31
9,4,8.0,813640.0,13.62


**3.3. Retrieve the total population and average per capita income for Catalonia for the years 2015, 2016, and 2017, along with their respective population growth and per capita income growth rates.**

**SQL Query Structure and Functions Utilized:**

- **Clauses**: SELECT, FROM, WITH, ORDER BY, ASC, DESC
- **Conditional Functions**: IFNULL()
- **Conversion Functions**: CAST()
- **Math Functions**: ROUND(), SUM()
- **Operators:** =, -, /, *
- **CTEs**: TotalCatalanPopulation, AveragePerCapitaIncome
- **Joins**: LEFT JOIN, JOIN

In [22]:
# Datasets: {mobile_data_cleaned}, {pop_density}, {percapita_income}

# SQL Query:
query = f"""
 -- CTE: Calculate annual population in Catalunya
WITH TotalCatalanPopulation AS (
    SELECT
        year,
        SUM(population) total_population
    FROM `{pop_density}`
    GROUP BY 1
    ORDER BY 1 ASC
),
 -- CTE: Calculate the average per capita income in Catalunya
AveragePerCapitaIncome AS (
    SELECT
        year,
        AVG(per_capita_income) avg_per_capita_income
    FROM `{percapita_income}`
    GROUP BY 1
    ORDER BY 1 ASC
)

SELECT
 p1.year,
 p1.total_population,
  -- Calculate annual population growth
 IFNULL(
     CAST(ROUND(((p1.total_population - p2.total_population) / p2.total_population * 100), 2) AS STRING),'No data'
    ) AS pop_growth,
 c1.avg_per_capita_income,
  -- Calculate annual per capita income growth
 IFNULL(
     CAST(ROUND(((c1.avg_per_capita_income - c2.avg_per_capita_income) / c2.avg_per_capita_income * 100), 2) AS STRING),'-'
    ) AS percapita_income_growth
FROM `TotalCatalanPopulation` p1

 -- LEFT JOIN to connect p1 (current year) to p2 ('+1' next year) to calculate the growth rates.
 -- This ensures all available data for the subsequent year (p2) is considered.

LEFT JOIN `TotalCatalanPopulation` p2
    ON p1.year = p2.year + 1
JOIN `AveragePerCapitaIncome` c1
    ON p1.year = c1.year
LEFT JOIN `AveragePerCapitaIncome` c2
    ON c1.year = c2.year + 1
ORDER BY 1 DESC
"""
# Execute the query
query_df(query)

,year,total_population,pop_growth,avg_per_capita_income,percapita_income_growth
0,2017,7971490,0.21,24774.5,1.79
1,2016,7954404,0.22,24338.5,2.44
2,2015,7937172,No data,23759.0,-


**3.4. Explore the annual performance trends for key indicators such as average signal strength, population, and per capita income across the provinces of Barcelona, Girona, Lleida, and Tarragona for the years 2016 and 2017.**

**SQL Query Structure and Functions Guidelines:**

- **Clauses**: SELECT, FROM, WITH, ORDER BY, ASC, DESC
- **Aggregate Functions**: COUNT(), AVG()
- **Math Functions**: ROUND()
- **Operators:** =, -, /, *
- **CTEs**: annual_data
- **Joins**: JOIN

In [64]:
# Datasets: {mobile_data_cleaned}, {pop_density}, {percapita_income}

# SQL Query:
query = f"""
 -- CTE: Retrieve the data for each year and province
WITH annual_data AS (
    SELECT
        m.year,
        m.province,
        ROUND(AVG(m.signal), 2) avg_signal,
        COUNT(*) activity_count,
        p.population,
        c.per_capita_income
    FROM `{mobile_data_cleaned}` m
    JOIN `{pop_density}` p
        ON m.province = p.province AND m.year = p.year
    JOIN `{percapita_income}` c
        ON m.province = c.province AND m.year = c.year
    GROUP BY 1, 2, 5, 6
    ORDER BY 2 ASC, 1 DESC
)

 -- Calculate annual growth percentages for each feature and province
SELECT
    a1.year,
    a1.province,
    a1.avg_signal,
     -- Calculate Average Signal Growth
    ROUND((a1.avg_signal - a2.avg_signal) / a2.avg_signal * 100, 2) AS avg_signal_growth,
     -- Calculate Activity Count Growth
    ROUND((a1.activity_count - a2.activity_count) / a2.activity_count * 100, 2) AS activity_count_growth,
     -- Calculate Population Growth
    ROUND((a1.population - a2.population) / a2.population * 100, 2) AS population_growth,
     -- Calculate Per Capita Income Growth
    ROUND((a1.per_capita_income - a2.per_capita_income) / a2.per_capita_income * 100, 2) AS per_capita_income_growth
FROM annual_data a1

 -- Join the 'annual_data' a2 to compare data for the same province and the next year (a2.year +1)
JOIN annual_data a2
    ON a1.province = a2.province
     -- Compare data for the current year with data from the next year
    AND a1.year = a2.year + 1 
ORDER BY a1.province ASC, a1.year DESC
"""

# Execute the query
query_df(query)

,year,province,avg_signal,avg_signal_growth,activity_count_growth,population_growth,per_capita_income_growth
0,2017,Barcelona,12.86,-3.89,-53.96,0.31,2.03
1,2016,Barcelona,13.38,-5.77,9.88,0.30,2.57
2,2017,Girona,11.92,-0.25,-51.11,0.06,1.54
3,2016,Girona,11.95,-1.89,-0.50,0.06,1.58
4,2017,Lleida,11.81,-7.95,-46.45,-0.03,1.82
5,2016,Lleida,12.83,2.72,-31.01,-0.01,2.87
6,2017,Tarragona,12.96,2.13,-39.88,-0.04,1.75
7,2016,Tarragona,12.69,-1.17,-18.64,-0.02,2.86


    - Check and compare annual activity_count (focus on high negative activity count growth in 2017)

In [24]:
# Datasets: {mobile_data_cleaned}, {pop_density}, {percapita_income}

# SQL Query:
query = f"""
SELECT
    year,
    COUNT(*) annual_activity_count
FROM `{mobile_data_cleaned}`
GROUP BY 1
ORDER BY 1 DESC
"""
# Execute the query
query_df(query)

,year,annual_activity_count
0,2017,2075215
1,2016,4311543
2,2015,4268655


    - Annual activity counts for the year 2017 show significantly fewer records compared to 2016 and 2015, which explains the negative growth observed in the previous query.

**3.5. We will now introduce simulated missing data into the 'pop_density' table by adding rows with NULL or different values in some of its columns, causing it to not perfectly align with the values in 'mobile_data_cleaned'.**

**This step enables us to observe the outcomes resulting from various types of joins (LEFT, RIGHT, INNER, FULL). Following this, a series of join operations will be performed to analyze the diverse outputs generated.**

    - Insert new data into 'pop_density' table

In [25]:
# Datasets: {mobile_data_cleaned}, {pop_density}, {percapita_income}

# SQL Query:
query = f"""
    INSERT INTO `{pop_density}`(year, population, sq_km)
    VALUES 
        (2015, 600000, 500),
        (2014, 530000, 490),
        (2016, 420000, 400),
        (2017, 385000, 380);
    
    INSERT INTO `{pop_density}`(province, year, sq_km)
    VALUES
        ('Costa Brava', 2017, 678),
        ('Pirineus', 2015, 10000);
    """
# Execute the query
run_query(query)  

Query successfully executed, and the table has been updated.


In [26]:
# Datasets: {mobile_data_cleaned}, {pop_density}, {percapita_income}

# SQL Query:
query = f"""   
    INSERT INTO `{pop_density}`(province, year, sq_km)
    VALUES
        ('Costa Brava', 2017, 678),
        ('Pirineus', 2015, 10000);
    """
# Execute the query
run_query(query)  

Query successfully executed, and the table has been updated.


    - Updated 'pop_density' table:

In [27]:
# Datasets: {mobile_data_cleaned}, {pop_density}, {percapita_income}

# SQL Query:
query = f"""
    SELECT *
    FROM `{pop_density}`;
    
    """
# Execute the query
query_df(query)  

,province,year,population,sq_km,density_per_sq_km
0,Girona,2016,766273.0,5908,129.7
1,Girona,2017,766705.0,5908,129.8
2,Girona,2015,765783.0,5908,129.6
3,Lleida,2015,742138.0,12172,61.0
4,Lleida,2016,742099.0,12172,61.0
5,Lleida,2017,741884.0,12172,61.0
6,Barcelona,2015,5618162.0,7726,727.2
7,Barcelona,2016,5635085.0,7726,729.4
8,Barcelona,2017,5652301.0,7726,731.6
9,Tarragona,2015,811089.0,6303,128.7


**3.5.1. FULL (OUTER) JOIN: Find matching and unique provinces between tables**

In [61]:
# Datasets: {mobile_data_cleaned}, {pop_density}, {percapita_income}

# SQL Query:
query = f"""
SELECT prov_pop_dens, prov_mob_data_cle
FROM (
     -- Select unique 'province' values in 'pop_density' table
    SELECT DISTINCT province AS prov_pop_dens
    FROM `{pop_density}`
) AS p
FULL OUTER JOIN (
     -- Select unique 'province' values in 'mobile_data_cleaned' table
    SELECT DISTINCT province AS prov_mob_data_cle
    FROM `{mobile_data_cleaned}`
) AS m ON p.prov_pop_dens = m.prov_mob_data_cle
ORDER BY 1 ASC
    """

# Execute the query
query_df(query)

,prov_pop_dens,prov_mob_data_cle
0,None,None
1,Barcelona,Barcelona
2,Costa Brava,None
3,Girona,Girona
4,Lleida,Lleida
5,Pirineus,None
6,Tarragona,Tarragona


This **FULL OUTER JOIN** or **FULL JOIN** shows all unique provinces from both tables, and in cases where a province exists in only one table, it pairs the corresponding entry with None in the other table to represent the absence of a matching province. This output illustrates the common and unique provinces between the two datasets.

**3.5.2. LEFT JOIN: Count the number of records for each province and year, using 'mobile_data_cleaned' as the left table and 'pop_density'**

In [39]:
# Datasets: {mobile_data_cleaned}, {pop_density}, {percapita_income}

# SQL Query:
query = f"""
    SELECT
      p.province,
      p.year,
      COUNT(m.signal) activity_count
    FROM `{mobile_data_cleaned}` m
    LEFT JOIN `{pop_density}` p
        ON p.province = m.province AND p.year = m.year
    GROUP BY 1,2
    ORDER BY 1,2 DESC;
    """
# Execute the query
query_df(query)  

,province,year,activity_count
0,Barcelona,2017,1415583
1,Barcelona,2016,3074699
2,Barcelona,2015,2798357
3,Girona,2017,269793
4,Girona,2016,551816
5,Girona,2015,554594
6,Lleida,2017,179247
7,Lleida,2016,334745
8,Lleida,2015,485195
9,Tarragona,2017,210592


The **LEFT JOIN** ensures that all records from the **left table (mobile_data_cleaned)** are retained in the output. It incorporates matches from the **right table (pop_density)** where available, while not displaying unmatched entries from the right table (for instance, None, Pirineus, and Costa Brava in this case).

In scenarios where the left table contains more 'provinces' or 'years' than the right table, unmatched entries from the right table would be filled with null or NaN values in the output.

**3.5.3. RIGHT JOIN: What is the average signal and maximum speed for each province and year, emphasizing 'pop_density' inclusion?**

In [34]:
# Datasets: {mobile_data_cleaned}, {pop_density}, {percapita_income}

# SQL Query:
query = f"""
    SELECT
      p.province,
      p.year,
      ROUND(AVG(m.signal),2) avg_signal,
      ROUND(MAX(m.speed),2) max_speed
    FROM `{mobile_data_cleaned}` m
    RIGHT JOIN `{pop_density}` p
        ON m.province = p.province AND m.year = p.year
    GROUP BY 1,2
    ORDER BY 1,2 DESC;
    """
# Execute the query
query_df(query) 

,province,year,avg_signal,max_speed
0,None,2017,NaN,NaN
1,None,2016,NaN,NaN
2,None,2015,NaN,NaN
3,None,2014,NaN,NaN
4,Barcelona,2017,12.86,254.8
5,Barcelona,2016,13.38,255.0
6,Barcelona,2015,14.20,255.0
7,Costa Brava,2017,NaN,NaN
8,Girona,2017,11.92,253.9
9,Girona,2016,11.95,255.0


A **RIGHT JOIN** ensures that all records from the **right table (pop_density)** are included, and matches from the **left table (mobile_data_cleaned)** are added where they exist. The None or NaN values in the output represent instances where provinces and years exist in the right table (in this case, None, Pirineus and Costa Brava) but don’t have corresponding matches in the left table, based on the specified conditions in the join.

**3.5.4. INNER JOIN: Calculate the average signal strength and maximum speed by matching province and year of both tables.**

In [35]:
# Datasets: {mobile_data_cleaned}, {pop_density}, {percapita_income}

# SQL Query:
query = f"""
    SELECT
      p.province,
      p.year,
      ROUND(AVG(m.signal),2) avg_signal,
      ROUND(MAX(m.speed),2) max_speed
    FROM `{pop_density}` p
    INNER JOIN `{mobile_data_cleaned}` m
        ON p.province = m.province AND p.year = m.year
    GROUP BY 1,2
    ORDER BY 1,2 DESC;
    """
# Execute the query
query_df(query)  

,province,year,avg_signal,max_speed
0,Barcelona,2017,12.86,254.8
1,Barcelona,2016,13.38,255.0
2,Barcelona,2015,14.20,255.0
3,Girona,2017,11.92,253.9
4,Girona,2016,11.95,255.0
5,Girona,2015,12.18,254.9
6,Lleida,2017,11.81,254.9
7,Lleida,2016,12.83,254.6
8,Lleida,2015,12.49,254.8
9,Tarragona,2017,12.96,254.9


An **INNER JOIN** retrieves only the rows from both tables that have matching values based on the specified join condition (p.province = m.province AND p.year = m.year).  Records where there are no matches based on the specified join condition are excluded from the result set.

**3.5.5. FULL JOIN: Calculate the average signal strength and maximum speed, merging province and year from both tables.**

In [37]:
# Datasets: {mobile_data_cleaned}, {pop_density}, {percapita_income}

# SQL Query:
query = f"""
    SELECT
      p.province,
      p.year,
      ROUND(AVG(m.signal),2) avg_signal,
      ROUND(MAX(m.speed),2) max_speed
    FROM `{pop_density}` p
    FULL JOIN `{mobile_data_cleaned}` m
        ON p.province = m.province AND p.year = m.year
    GROUP BY 1,2
    ORDER BY 1,2 DESC;
    """
# Execute the query
query_df(query)    

,province,year,avg_signal,max_speed
0,None,2017,NaN,NaN
1,None,2016,NaN,NaN
2,None,2015,NaN,NaN
3,None,2014,NaN,NaN
4,Barcelona,2017,12.86,254.8
5,Barcelona,2016,13.38,255.0
6,Barcelona,2015,14.20,255.0
7,Costa Brava,2017,NaN,NaN
8,Girona,2017,11.92,253.9
9,Girona,2016,11.95,255.0


Again, a **FULL JOIN** retrieves all the rows from both tables, joining the rows where the specified conditions are met and including rows that do not have corresponding matches in the other table. The None values and NaN in the output represent the instances where provinces and years exist in one table but not in the other. 

    - Delete added rows

In [65]:
# Datasets: {mobile_data_cleaned}, {pop_density}, {percapita_income}

# SQL Query:
query = f"""
    DELETE FROM `{pop_density}`
    WHERE 
        province IS NULL 
        OR province IN ('Costa Brava', 'Pirineus');
    """
# Execute the query
run_query(query)    

Query successfully executed, and the table has been updated.


#### 8. Case Statements

**8.1. Update the 'signal' column in the '{mobile_2015_2017}' table, setting it to 'Above avg' when the 'signal' is above 14, 'On Avg' when between 13 and 14, and 'Under Avg' when below 13.**

In [66]:
# Datasets: {mobile_data_cleaned}, {pop_density}, {percapita_income}

# SQL Query: max, min and average
query = f"""
    SELECT
      max(signal) max_signal,
      min(signal) min_signal,
      avg(signal) avg_signal
    FROM `{mobile_data_cleaned}`
    """
# Execute the query
query_df(query)   

,max_signal,min_signal,avg_signal
0,99,0,13.215715


    - Create a new column to store the updated value

In [67]:
# Datasets: {mobile_data_cleaned}, {pop_density}, {percapita_income}

# SQL Query:
query = f"""
    ALTER TABLE `{mobile_data_cleaned}`
    ADD COLUMN signal_eval STRING;
    """

# Execute the query
run_query(query)    

Query successfully executed, and the table has been updated.


    - Update values

**SQL Query Structure and Functions Utilized:**

- **Clauses**: UPDATE, SET, WHERE, IS NULL 
- **Conditional Functions**: CASE, WHEN, THEN, ELSE, END
- **Operators**: =, <, BETWEEN, >


In [69]:
# Datasets: {mobile_data_cleaned}, {pop_density}, {percapita_income}

# SQL Query:
query = f"""
    UPDATE `{mobile_data_cleaned}`
    SET signal_eval =
        CASE
            WHEN signal < 13 THEN 'Under Avg'
            WHEN signal BETWEEN 13 AND 14 THEN 'On Avg'
            WHEN signal > 14 THEN 'Above Avg'
            END
        WHERE signal_eval IS NULL;
    """

# Execute the query and store the result in the DataFrame
run_query(query)    

Query successfully executed, and the table has been updated.


    - Check output

In [70]:
# Datasets: {mobile_data_cleaned}, {pop_density}, {percapita_income}

# SQL Query:
query = f"""
    SELECT
      signal,
      signal_eval
    FROM `{mobile_data_cleaned}`
    LIMIT 100
    """
# Execute the query
query_df(query) 

,signal,signal_eval
0,6,Under Avg
1,11,Under Avg
2,24,Above Avg
3,17,Above Avg
4,13,On Avg
...,...,...
95,21,Above Avg
96,14,On Avg
97,14,On Avg
98,18,Above Avg


    - Delete 'signal_eval' column

In [71]:
# Datasets: {mobile_data_cleaned}, {pop_density}, {percapita_income}

# Delete the added column
query = f"""
    ALTER TABLE `{mobile_data_cleaned}`
    DROP COLUMN signal_eval;
    """

# Execute the query
run_query(query)    

Query successfully executed, and the table has been updated.


#### 9. Statistical Aggregate Functions

4. Calculate the correlation coefficient between the average signal strength (signal) and population density for all provinces.


In [ ]:
# Datasets: {mobile_data_cleaned}, {pop_density}, {percapita_income}

query = f"""
WITH avg_signal_table AS (
    SELECT
        m.year,
        m.province,
        AVG(m.signal) avg_signal
    FROM
        `{mobile_data_cleaned}` m
    GROUP BY m.year, m.province
)

SELECT 
    # Calculate Pearson Correlation
    CORR(t.avg_signal, p.density_per_sq_km) corr_signal_density
FROM avg_signal_table t
JOIN `{pop_density}` p 
  ON t.province = p.province AND t.year = p.year
;
"""
# Execute the query
query_df(query) 

    -  This value suggests a moderately strong positive linear correlation between the average signal strength (avg_signal) and population density (density_per_sq_km) for the given provinces and years in the dataset. A correlation coefficient of 0.71 indicates that as population density increases, the average signal strength tends to increase as well.

5. Calculate the correlation coefficient between the average signal strength (signal) and per capita income for all provinces in the year 2017.


In [ ]:
# Datasets: {mobile_data_cleaned}, {pop_density}, {percapita_income}

query = f"""
WITH avg_signal_table AS (
    SELECT
        m.year,
        m.province,
        AVG(m.signal) avg_signal
    FROM
        `{mobile_data_cleaned}` m
    GROUP BY m.year, m.province
)

SELECT 
    CORR(t.avg_signal, c.per_capita_income) corr_signal_income
FROM avg_signal_table t
JOIN `{percapita_income}` c 
  ON t.province = c.province AND t.year = c.year
;
"""
# Execute the query
query_df(query)  

    - This value suggests a positive linear correlation too, but it is weaker than the correlation between average signal strength (avg_signal) and population density (density_per_sq_km) that was calculated in the previous query.

1. Calculate the Pearson correlation coefficient between different features pairs in the '{mobile_2015_2017}' table for all years in the dataset.

In [ ]:
# Datasets: {mobile_data_cleaned}, {pop_density}, {percapita_income}

query = f"""
    SELECT
      CORR(signal, speed) AS signal_speed,
      CORR(signal, hour_24h) AS signal_hour,
      CORR(signal, CAST(postal_code AS INT64)) AS signal_postalcode,
      CORR(lat, long) AS lat_long,
      CORR(precission, satellites) AS precision_satellites,
      CORR(precission, speed) AS precision_speed,
      CORR(precission, signal) AS precision_signal,
      CORR(precission, CAST(postal_code AS INT64)) AS precission_postalcode,
      CORR(satellites, CAST(postal_code AS INT64)) AS satellites_postalcode
    FROM `{mobile_data_cleaned}`
    """
# Execute the query
query_df(query)  

2. Calculate the covariance between 'signal' strength and 'speed' in the '{mobile_2015_2017}' table.

In [ ]:
# Datasets: {mobile_data_cleaned}, {pop_density}, {percapita_income}

query = f"""
    SELECT
      COVAR_POP(signal, speed) AS COVAR_signal_speed,
      CORR(signal, speed) AS CORR_signal_speed
    FROM `{mobile_data_cleaned}`
    """
# Execute the query
query_df(query)    

    - Covariance measures the degree to which two variables change together. In this case, the negative value of -0.95 suggests that there is a negative linear relationship between "signal" and "speed." When one variable increases, the other tends to decrease, and vice versa. However, the correlation coefficient (that unlike covariance, the correlation coefficient is a standardized measure within the range of -1 to 1.), which is approximately -0.004, indicates an extremely weak linear relationship, close to zero, suggesting that there is no substantial linear correlation between these two variables. In essence, while there is a negative covariance, the correlation coefficient underscores the absence of a strong linear connection between "signal" and "speed" in the dataset.

3. Calculate the standard deviation of 'signal' strength in the '{mobile_2015_2017}' table for the year 2017.

In [ ]:
# Datasets: {mobile_data_cleaned}, {pop_density}, {percapita_income}

query = f"""
    SELECT
      ROUND(STDDEV_POP(signal),2) AS STD_signal,
      ROUND(MIN(signal),2) AS MIN_signal,
      ROUND(MAX(signal),2) AS MAX_signal,
      ROUND(AVG(signal),2) AS AVG_signal      
    FROM `{mobile_data_cleaned}`
    WHERE year = 2017
    """
# Execute the query
query_df(query)

    - These statistics provide insights into the variability and range of signal strengths in the dataset for the year 2017. The standard deviation of 7.18 indicates that signal strengths vary from the average by an average of approximately 7.18 units, and the signal strength ranged from 0.0 to 65.0 during that year.

4. Calculate the variance of 'speed' in the '{mobile_2015_2017}' table for the year 2017.

In [ ]:
# Datasets: {mobile_data_cleaned}, {pop_density}, {percapita_income}

query = f"""
    SELECT
      VAR_POP(speed) VAR_speed,
      STDDEV_POP(speed) AS STD_speed,
      MAX(speed) - MIN(speed) AS Range_speed
    FROM `{mobile_data_cleaned}`
    WHERE year=2017
    """
# Execute the query
query_df(query)  

    - The statistics for the 'speed' variable in the year 2017 reveal a substantial degree of variability, as evidenced by the high population variance, standard deviation, and range. This variation suggests that the data points for 'speed' exhibit a wide dispersion from the mean and cover a significant range, reflecting diverse speed measurements throughout the year 2017.

#### 10. Window Functions

In [ ]:
# Datasets: {mobile_data_cleaned}

query = f"""
    SELECT
        province,
        date AS first_activity_date,
        hour AS first_activity_hour,
        FIRST_VALUE(date) OVER (PARTITION BY province ORDER BY date, hour) AS first_net_provider,
        date AS last_activity_date,
        hour AS last_activity_hour,
        LAST_VALUE(date) OVER (PARTITION BY province ORDER BY date, hour) AS last_net_provider,
    FROM `{mobile_data_cleaned}`
    WHERE net_provider IS NOT NULL
    """
# Execute the query
query_df(query)


Before we start working with window functions, we will set the stage by inserting sample data into the {mobile_data_cleaned} table. This data will help us explore the distinctions between the RANK() and DENSE_RANK() window functions when ranking operators within networks.

Additionally, we will create a **view**, {netwop_4G_view}, to filter and extract a specific subset of the data, focusing on '4G' networks. This view will serve as the basis for our exploration of window functions, allowing us to rank operators and gain insights into their network activity.

    - Insert new rows

In [ ]:
# Datasets: {mobile_data_cleaned}, {pop_density}, {percapita_income}

query = f"""
    INSERT INTO `{mobile_data_cleaned}` (network, operator, net, signal, speed)
    VALUES
        ('invented_network', 'invented_operator', '4G', 11, 13),
        ('invented_network', 'invented_operator', '4G', 12, 14),
        ('invented_network', 'invented_operator2', '4G', 12, 14)
    """

# Execute the query
run_query(query)  

In [ ]:
# Datasets: {mobile_data_cleaned}, {pop_density}, {percapita_income}

query = f"""
    SELECT
        *
    FROM `{mobile_data_cleaned}`
    WHERE REGEXP_CONTAINS(operator,'invented')
    """

# Execute the query
query_df(query)  

    - Create View

In [ ]:
# Network & Operator (unique pairs) and 4G net table (2015_2017) path 
netwop_4G_view = "bq-analyst-230590.project_cat_mobile_coverage_2015_2017.netw_op_4G_view"

In [ ]:
# Datasets: {mobile_data_cleaned}, {pop_density}, {percapita_income}, 
# Views: {netwop_4G_view}

query = f"""
    CREATE OR REPLACE VIEW `{netwop_4G_view}` AS
    SELECT
      network,
      operator,
      COUNT(*) AS activity_count,
      ROUND(AVG(signal), 2) AS avg_signal,
      ROUND(AVG(speed), 2) AS avg_speed
    FROM `{mobile_data_cleaned}`
    WHERE net = '4G'
    GROUP BY network, operator
    ORDER BY network
    """

# Execute the query
run_query(query)  

    - Output

In [ ]:
# Datasets: {mobile_data_cleaned}, {pop_density}, {percapita_income}, 
# Views: {netwop_4G_view}

query = f"""
    SELECT
      *
    FROM `{netwop_4G_view}`
    """

# Execute the query
query_df(query)  

1. How do networks within the '4G' network rank based on the number of operators, and which networks are at the top 10 in terms of operator count?

In [ ]:
# Datasets: {mobile_data_cleaned}, {pop_density}, {percapita_income}, 
# Views: {netwop_4G_view}

# Rank Vs Dense Rank

query = f"""
SELECT
  network,
  COUNT(operator) operator_count,
  RANK() OVER(ORDER BY COUNT(operator) DESC) overall_rank_num_op,
  DENSE_RANK() OVER(ORDER BY COUNT(operator) DESC) overall_DENSE_rank_num_op
FROM `{netwop_4G_view}`
GROUP BY 1
ORDER BY 2 DESC
LIMIT 10
    """

# Execute the query
query_df(query)  

    - Drop 'invented' rows from original dataset

In [ ]:
# Datasets: {mobile_data_cleaned}, {pop_density}, {percapita_income}, 
# Views: {netwop_4G_view}

query = f"""
    DELETE FROM `{mobile_data_cleaned}`
    WHERE REGEXP_CONTAINS(operator,'invented')
    """

# Execute the query
run_query(query)  

2. Within '4G' networks with multiple operators, how are operators ranked based on their activity count, and what is the comparative distribution of their rankings within their respective networks?

In [ ]:
# Datasets: {mobile_data_cleaned}, {pop_density}, {percapita_income}, 
# Views: {netwop_4G_view}

query = f"""
WITH FilteredNetworks AS (
  SELECT
    network
  FROM `{netwop_4G_view}`
  GROUP BY network
  HAVING COUNT(DISTINCT operator) > 1
)

SELECT
  network,
  operator,
  activity_count,
  RANK() OVER (PARTITION BY network ORDER BY activity_count) AS rank_within_network,
  PERCENT_RANK() OVER (PARTITION BY network ORDER BY activity_count) AS percent_rank,
  DENSE_RANK() OVER (PARTITION BY network ORDER BY activity_count) AS dense_rank
FROM `{netwop_4G_view}`
WHERE network IN (SELECT network FROM FilteredNetworks)
    """

# Execute the query
query_df(query)  

3. How does the moving average signal strength within the 'Orange' network for each operator compare to their respective average signal strength?

In [ ]:
# Datasets: {mobile_data_cleaned}, {pop_density}, {percapita_income}, 
# Views: {netwop_4G_view}

query = f"""
SELECT
  network,
  operator,
  avg_signal,
  AVG(avg_signal) OVER (PARTITION BY network ORDER BY avg_signal ROWS BETWEEN 2 PRECEDING AND 2 FOLLOWING) AS moving_avg_signal
FROM `{netwop_4G_view}`
WHERE network = 'Orange'
GROUP BY 1,2,3
ORDER BY 1, 2 ASC
    """

# Execute the query
query_df(query)

    - To calculate the moving average, it considers the signal strength values of each operator and computes the average of the surrounding values within a window of two preceding and two following rows (including the operator's value). This provides an estimate of signal strength variations for each operator within the network, which can be useful for identifying trends and deviations in signal strength over time.

4. Can we divide operators in the '4G' network into quartiles based on their average signal strength, and what insights can we gain into the performance variation within each quartile?

In [ ]:
# Datasets: {mobile_data_cleaned}, {pop_density}, {percapita_income}, 
# Views: {netwop_4G_view}

query = f"""
WITH Quartiles AS (
    SELECT
        operator,
        avg_signal,
        # Divide operators into quartiles (1-4), with 1 being the lowest and 4 being the highest signal strength quartile.
        NTILE(4) OVER (PARTITION BY network ORDER BY avg_signal) AS signal_quartile 
    FROM `{netwop_4G_view}`
    )

SELECT
  signal_quartile,
  COUNT(operator) AS operator_count,
  MIN(avg_signal) AS min_signal,
  MAX(avg_signal) AS max_signal,
  AVG(avg_signal) AS avg_signal
FROM Quartiles
GROUP BY signal_quartile
ORDER BY signal_quartile
    """

# Execute the query
query_df(query)

In [ ]:
# Datasets: {mobile_data_cleaned}, {pop_density}, {percapita_income}, 
# Views: {netwop_4G_view}

query = f"""
WITH Quartiles AS (
    SELECT
        operator,
        avg_signal,
        # Divide operators into quartiles (1-4), with 1 being the lowest and 4 being the highest signal strength quartile.
        NTILE(4) OVER (PARTITION BY network ORDER BY avg_signal) AS signal_quartile 
    FROM `{netwop_4G_view}`
    )

SELECT
  signal_quartile,
  COUNT(operator) AS operator_count,
  MIN(avg_signal) AS min_signal,
  MAX(avg_signal) AS max_signal,
  AVG(avg_signal) AS avg_signal
FROM Quartiles
GROUP BY signal_quartile
ORDER BY signal_quartile
    """

# Execute the query
query_df(query)

    - This analysis can be valuable for assessing the performance variation among different telecommunication providers (networks). It helps in understanding how the operators within each provider compare in terms of signal strength. For example, Quartile 4 includes operators with the highest signal strength, while Quartile 1 includes operators with the lowest signal strength. Operators in Quartiles 2 and 3 have signal strengths that fall in between.

    - This information can be useful for network administrators and planners who are interested in evaluating signal strength across various telecommunication providers, which enables them to make informed decisions regarding network optimization and performance improvements.

5. What insights can we gain about signal strength changes and network speed variations among operators in the '4G' network, especially within the 'Orange' network category, using LEAD and LAG window functions without considering time-based changes?

In [ ]:
# Datasets: {mobile_data_cleaned}, {pop_density}, {percapita_income}, 
# Views: {netwop_4G_view}

query = f"""
SELECT
    network,
    operator,
    avg_signal,
    LEAD(avg_signal) OVER (PARTITION BY network ORDER BY avg_signal) lead_avg_signal
    FROM `{netwop_4G_view}`
WHERE network = 'Orange'
GROUP BY 1,2,3
ORDER BY 3 ASC
    """

# Execute the query
query_df(query)

In [ ]:
# Datasets: {mobile_data_cleaned}, {pop_density}, {percapita_income}, 
# Views: {netwop_4G_view}

query = f"""
SELECT
    network,
    operator,
    avg_speed,
    # LAG( ,3) takes the avg_speed of row_num - 3.
    LAG(avg_speed,3) OVER(PARTITION BY network ORDER BY avg_speed) lag_avg_speed
FROM `{netwop_4G_view}`
WHERE network = 'Orange'
GROUP BY 1,2,3
ORDER BY 3 ASC
    """

# Execute the query
query_df(query)

    - 6. AAmong 'Orange' operators in the '4G' network, how does their ranking based on signal strength compare to their ranking based on the number of records? Do 'Orange' operators with strong signal strength also have a larger volume of records, and does this impact their overall ranking?

In [ ]:
# Datasets: {mobile_data_cleaned}, {pop_density}, {percapita_income}, 
# Views: {netwop_4G_view}

query = f"""
SELECT
    operator,
    avg_signal AS signal_strength,
    ROW_NUMBER() OVER (PARTITION BY network ORDER BY avg_signal DESC) AS signal_rank,
    activity_count AS record_count,
    ROW_NUMBER() OVER (PARTITION BY network ORDER BY activity_count DESC) AS record_rank
FROM `{netwop_4G_view}`
WHERE network = 'Orange'
ORDER BY signal_rank;
    """

# Execute the query
query_df(query)

    - We can observe that the ranking based on signal strength doesn't necessarily correlate with the ranking based on the number of records. Operators like RACC, with the third-highest signal strength, are ranked seventh (out of nine) in terms of record count, indicating a low presence in the network.

### 11. Dates

Date range

In [ ]:
# SQL query
query = f"""
    SELECT
        MIN(date) AS first_date_recorded,
        MAX(date) AS last_date_recorded,
        DATE_DIFF(MAX(date), MIN(date), DAY) AS total_days_recorded
    FROM `{mobile_data_cleaned}`
    """
# Execute the query
query_df(query) 

Top 10 dates with the highest activity

In [ ]:
# SQL query
query = f"""
    SELECT
      date,
      COUNT(*) AS record_count
    FROM `{mobile_data_cleaned}`
    GROUP BY date
    ORDER BY record_count DESC
    LIMIT 10
    """
# Execute the query
query_df(query) 

Monthly Activity Rank within Quarters and Across the Year

In [ ]:
# SQL query
query = f"""
    WITH MonthlyCounts AS (
        SELECT
            EXTRACT(QUARTER FROM date) AS quarter,
            EXTRACT(MONTH FROM date) AS month,
            COUNT(*) AS record_count
        FROM `{mobile_data_cleaned}`
        GROUP BY quarter, month
    )

    SELECT
      quarter,
      month,
      record_count,
      RANK() OVER (ORDER BY record_count DESC) AS month_rank
    FROM MonthlyCounts
    ORDER BY quarter, record_count DESC;
    """
# Execute the query and store the result in the DataFrame
month_rank_df = query_df(query) 

month_rank_df

In [ ]:
# Create a custom color palette for each month within a quarter
palette = sns.color_palette("tab10", n_colors=12)  # Use n_colors=12 for 12 months

# Pivot the DataFrame to have months as columns for stacking
stacked_df = month_rank_df.pivot(index='quarter', columns='month', values='record_count')

# Bar plot with stacked bars for each quarter and different colors for months
plt.figure(figsize=(12, 6))
ax = stacked_df.plot(kind='bar', stacked=True, color=palette)
plt.title('Stacked Bar Plot of Quarter-wise Record Count')
plt.xlabel('Quarter')
plt.ylabel('Record Count')
plt.legend(title='Month', loc='upper right', bbox_to_anchor=(1.15, 1))
plt.ticklabel_format(style='plain', axis='y')  # Disable scientific notation on y-axis
plt.show()


In [ ]:
# Define custom color palettes for each quarter
colors_first_quarter = sns.color_palette("Blues", n_colors=3)  # Blue colors for months 1, 2, 3
colors_second_quarter = sns.color_palette("Greens", n_colors=3)  # Green colors for months 4, 5, 6
colors_third_quarter = sns.color_palette("Oranges", n_colors=3)  # Orange colors for months 7, 8, 9
colors_fourth_quarter = sns.color_palette("Purples", n_colors=3)  # Purple colors for months 10, 11, 12

# Pivot the DataFrame to have months as columns for stacking
stacked_df = month_rank_df.pivot(index='quarter', columns='month', values='record_count')

# Create a stacked bar plot with distinct colors for each month within a quarter
plt.figure(figsize=(12, 6))

# Loop through each quarter and plot stacked bars with the custom color palette
quarters = stacked_df.index
for quarter, color_palette in zip(quarters, [colors_first_quarter, colors_second_quarter, colors_third_quarter, colors_fourth_quarter]):
    quarter_data = stacked_df.loc[quarter]
    quarter_data.plot(kind='bar', stacked=True, color=color_palette, label=f'Quarter {quarter}')

# Calculate the overall average for all values
overall_average = stacked_df.stack().mean()

# Plot a red horizontal line for the overall average
plt.axhline(y=overall_average, color='red', linestyle='--', label=f'Avg. Overall')

plt.title('Stacked Bar Plot of Quarter-wise Record Count with Distinct Colors for Months')
plt.xlabel('Month')
plt.ylabel('Record Count')
plt.legend(title='Legend', loc='upper right', bbox_to_anchor=(1.15, 1))
plt.ticklabel_format(style='plain', axis='y')  # Disable scientific notation on y-axis
plt.show()


### Hours

Hourly Period Counts

In [ ]:
# SQL query
query = f"""
    SELECT
      CASE
        WHEN EXTRACT(HOUR FROM hour) BETWEEN 0 AND 6 THEN 'Dawn'
        WHEN EXTRACT(HOUR FROM hour) BETWEEN 7 AND 12 THEN 'Morning'
        WHEN EXTRACT(HOUR FROM hour) BETWEEN 13 AND 18 THEN 'Afternoon'
        ELSE 'Night'
      END AS period,
      COUNT(*) AS record_count
    FROM `{mobile_data_cleaned}`
    GROUP BY 1
    ORDER BY 2 DESC
    """

# Execute the query and store the result in the DataFrame
hourly_period_counts = query_df(query)

hourly_period_counts

In [ ]:
import plotly.express as px

# Define the custom order and colors
custom_order = ["Dawn", "Morning", "Afternoon", "Night"]

# Create an interactive pie chart with Plotly
fig = px.pie(hourly_period_counts, values='record_count', names='period', 
             title='Hourly Period Counts', 
             hover_data=['record_count'], 
             labels={'record_count': 'Record Count'},
             category_orders={"period": custom_order})

# Customize the layout (optional)
fig.update_traces(textinfo='percent+label', pull=[0.1, 0.1, 0.1, 0.1])

# Show the chart
fig.show()

# Save the plot as an HTML file in the 'Python Plots' folder
plot_file_path = "Python Plots/hourly_period_counts.html"
fig.write_html(plot_file_path)

[View the Plot](./Python%20Plots/hourly_period_counts.html)

### Towns and Province activity

Top 50 Towns with the Highest Mobile Activity Recorded

In [ ]:
# SQL query
query = f"""
    SELECT
      postal_code,
      town_name,
      COUNT(*) AS record_count
    FROM
      `{mobile_data_cleaned}`
    WHERE postal_code IS NOT NULL
    GROUP BY postal_code, town_name
    ORDER BY 3 DESC
    LIMIT 50
    """

# Execute the query
query_df(query)

In [ ]:
# SQL query
query = f"""
WITH ProvinceActivity AS (
  SELECT
      CASE
        WHEN LEFT(CAST(postal_code AS STRING), 2) = '08' THEN 'Barcelona'
        WHEN LEFT(CAST(postal_code AS STRING), 2) = '25' THEN 'Lleida'
        WHEN LEFT(CAST(postal_code AS STRING), 2) = '17' THEN 'Girona'
        WHEN LEFT(CAST(postal_code AS STRING), 2) = '43' THEN 'Tarragona'
        ELSE 'Not defined'
      END AS province,
      COUNT(*) AS record_count
  FROM `{mobile_data_cleaned}`
  GROUP BY province
)

SELECT
    province,
    record_count,
    ROUND((record_count / SUM(record_count) OVER ()) * 100,2) AS percentage
FROM ProvinceActivity
ORDER BY 2 DESC;
    """
# Execute the query
query_df(query)

### Network, operators and signal

In [ ]:
# SQL query
query = f"""
SELECT
  network,
  operator,
  COUNT(*) record_count,
  ROUND(AVG(signal),1) avg_netw_signal
FROM
  `{mobile_data_cleaned}`
GROUP BY 1,2
ORDER BY 3 DESC;
    """

# Execute the query and store the result in the DataFrame
netw_oper_signal = query_df(query)

netw_oper_signal

In [ ]:
networks = netw_oper_signal['network'].nunique()
operators = netw_oper_signal['operator'].nunique()

display(Markdown(f"There are {networks} unique networks and {operators} distinct operators"))

Which (national) network has the highest activity recorded?

In [ ]:
# Group the DataFrame by the "network" column and calculate the total activity
network_activity = netw_oper_signal.groupby('network')['record_count'].sum().reset_index()

# Find the network with the highest total activity
max_activity_network = network_activity[network_activity['record_count'] == network_activity['record_count'].max()]

# Print the network with the highest total activity
max_activity_network

And the operator?

In [ ]:
# Group the DataFrame by the "operator" column and calculate the total activity
operator_activity = netw_oper_signal.groupby('operator')['record_count'].sum().reset_index()

# Find the operator with the highest total activity
max_activity_operator = operator_activity[operator_activity['record_count'] == operator_activity['record_count'].max()]

# Print the operator with the highest total activity
max_activity_operator

Which network/operator has the highest average signal?

In [ ]:
# Find the index of the row with the maximum "avg_netw_signal"
max_signal_index = netw_oper_signal['avg_netw_signal'].idxmax()

# Retrieve the corresponding row with the maximum "avg_netw_signal"
top_signal = netw_oper_signal.loc[max_signal_index]

# Display the row with the maximum "avg_netw_signal"
top_signal

Which network/operator has the highest average signal among the top 25% of recorded activities?

In [ ]:
# Calculate the threshold for the top 25% percentile of "record_count"
threshold = netw_oper_signal['record_count'].quantile(0.75)

# Filter the DataFrame to select rows with "record_count" greater than or equal to the threshold
top_25_percentile = netw_oper_signal[netw_oper_signal['record_count'] >= threshold]

# Find the row with the maximum "avg_netw_signal" within the filtered DataFrame
max_avg_netw_signal_row = top_25_percentile.loc[top_25_percentile['avg_netw_signal'].idxmax()]

# Display the row with the maximum "avg_netw_signal"
max_avg_netw_signal_row

Top Signal Strength by Operator and Net_Provider (top 10% record_count, excluding null values)

In [ ]:
# SQL query: Top Signal Strength by Operator and Network (top 10% record_count, excluding null values)
query = f"""
WITH Top10Percent AS (
  
  # Subquery to prepare data to calculate the number of operators for each network
  SELECT
    net_provider,
    num_operator,
    record_count,
    RANK() OVER (ORDER BY record_count DESC) AS record_count_rank,
    avg_signal,
    quartile
  FROM (
    SELECT
      LOWER(net_provider) AS net_provider,
      
      # Calculate the number of distinct operators for each network
      COUNT(DISTINCT LOWER(operator)) AS num_operator,
      
      COUNT(*) AS record_count,
      ROUND(AVG(signal), 1) AS avg_signal,
      NTILE(10) OVER (ORDER BY COUNT(*) DESC) AS quartile
    FROM
      `{mobile_data_cleaned}`
    WHERE
    
    # Exclude rows with network 'null' (stored as string)
      net_provider != 'null'
    GROUP BY 1
  )
)

SELECT
  net_provider,
  num_operator,
  record_count,
  record_count_rank,
  avg_signal
FROM
  Top10Percent
WHERE
  quartile = 1
ORDER BY
  avg_signal DESC;
    """
# Execute the query
query_df(query)

Top Network and Operator by Record Count for Each Province

In [ ]:
# SQL query
query = f"""
WITH ProvinceActivity AS (
  SELECT
    CASE
      WHEN LEFT(CAST(postal_code AS STRING), 2) = '08' THEN 'Barcelona'
      WHEN LEFT(CAST(postal_code AS STRING), 2) = '25' THEN 'Lleida'
      WHEN LEFT(CAST(postal_code AS STRING), 2) = '17' THEN 'Girona'
      WHEN LEFT(CAST(postal_code AS STRING), 2) = '43' THEN 'Tarragona'
      ELSE 'Not defined'
    END AS province,
    LOWER(network) AS network,
    LOWER(operator) AS operator,
    COUNT(*) AS record_count
  FROM `{mobile_data_cleaned}`
  GROUP BY province, network, operator
)

# Find the Rank of Network and Operator by Record Count for Each Province
, RankedNetwork AS (
  SELECT
    province,
    network,
    operator,
    record_count,
    RANK() OVER (PARTITION BY province ORDER BY record_count DESC) AS network_rank
  FROM ProvinceActivity
)

, RankedOperator AS (
  SELECT
    province,
    network,
    operator,
    record_count,
    RANK() OVER (PARTITION BY province ORDER BY record_count DESC) AS operator_rank
  FROM ProvinceActivity
)

# Select the Top Network and Top Operator for Each Province
SELECT
  p.province,
  n.network AS top_network,
  o.operator AS top_operator
  #n.record_count AS network_record_count,
  #o.record_count AS operator_record_count
FROM RankedNetwork n
JOIN RankedOperator o ON n.province = o.province AND n.network_rank = 1 AND o.operator_rank = 1
JOIN (SELECT DISTINCT province FROM ProvinceActivity) p ON n.province = p.province
ORDER BY p.province;
    """
# Execute the query
query_df(query)

### Description and activity

Description:
    
    - STATE_IN_SERVICE (0),
    - STATE_OUT_OF_SERVICE (1),
    - STATE_EMERGENCY_ONLY (2),
    - STATE_POWER_OFF (3)
    
User Activity:

    - IN_VEHICLE
    - STILL
    - ON_FOOT
    - TILTING
    - UNKNOWN
    - ON_BICYCLE

Activity Rank by Description with Overall Rank

In [ ]:
# SQL query
query = f"""
# Activity Rank by Description with Overall Rank
WITH ActivityRank AS (
  SELECT
    description,
    activity,
    RANK() OVER (PARTITION BY description ORDER BY COUNT(*) DESC) AS rank_by_description,
    COUNT(*) AS record_count
  FROM
    `{mobile_data_cleaned}`
  GROUP BY 1, 2
)

SELECT
  description,
  activity,
  rank_by_description,
  #record_count,
  RANK() OVER (ORDER BY rank_by_description, record_count DESC) AS overall_rank
FROM ActivityRank
ORDER BY 1, 4 ASC;
    """
# Execute the query
query_df(query)

Calculate activity rank by description and activity, including most frequent town and overall rank

In [ ]:
# SQL query
query = f"""
# Subquery to calculate the rank of activities by description
WITH ActivityRank AS (
  SELECT
    description,
    activity,
    RANK() OVER (PARTITION BY description ORDER BY COUNT(*) DESC) AS rank_by_description,
    COUNT(*) AS record_count
  FROM
    `bigquery-public-data.catalonian_mobile_coverage_eu.mobile_data_2015_2017`
  GROUP BY description, activity
),

# Subquery to find the most frequent town for each unique pair of description and activity
MostFrequentTown AS (
  SELECT
    description,
    activity,
    town_name,
    RANK() OVER (PARTITION BY description, activity ORDER BY COUNT(*) DESC) AS town_rank
  FROM
    `{mobile_data_cleaned}`
  GROUP BY description, activity, town_name
)

# Main query to combine the results and calculate overall rank
SELECT
  ar.description,
  ar.activity,
  ar.rank_by_description,
  RANK() OVER (ORDER BY ar.rank_by_description, ar.record_count DESC) AS overall_rank,
  mft.town_name AS most_frequent_town
FROM ActivityRank ar
# Join with the MostFrequentTown subquery to include the most frequent town
JOIN MostFrequentTown mft ON ar.description = mft.description AND ar.activity = mft.activity
# Filter for the most frequent town
WHERE mft.town_rank = 1 
ORDER BY 1, 3 ASC;
    """
# Execute the query
query_df(query)